# Classification - Workshop

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from astroML.datasets import fetch_rrlyrae_combined
from astroML.utils import split_samples
from astroML.utils import completeness_contamination

## Exercise 1: Classify AGN objects

AGN classification , BPT , https://ned.ipac.caltech.edu/level5/Glossary/Essay_bpt.html

### Setting up the data

In [57]:
# CREATING DATA STRUCTURE

# > Loading the emission line data from the MPA-JHU team's 'galSpec' table for the SDSS galaxies:

PATH_AGN_data = "data/sdss_lines.csv"

data = np.genfromtxt(PATH_AGN_data, delimiter=",")
# The data file is organized in 452 lines (i.e. different objects), and 15 columns

# To check file dimensions:
# print(data.shape)

ID             = data[:,0]  # object ID
H_BETA_FLUX    = data[:,1]  # flux of H_beta line
OIII_5007_FLUX = data[:,3]  # flux of OIII line at 5007 Amstrong
H_ALPHA_FLUX   = data[:,7]  # flux of H_alpha line
NII_6584_FLUX  = data[:,9]  # flux of NII line at 6584 Amstrong
SII_6717_FLUX  = data[:,12] # flux of SII line at 6717 Amstrong

labels = np.genfromtxt(PATH_AGN_data, delimiter=',', usecols=-2, dtype=str)
# reading labels from second column from the last

# > Replacing negative (bogus) fluxes with sample median:
#   NOTE: This will introduce some noise in the data, which is good for the sake of this exercise
OIII_5007_FLUX[OIII_5007_FLUX <= 0] = np.median(OIII_5007_FLUX)
NII_6584_FLUX[NII_6584_FLUX   <= 0] = np.median(NII_6584_FLUX)
SII_6717_FLUX[SII_6717_FLUX   <= 0] = np.median(SII_6717_FLUX)
H_BETA_FLUX[H_BETA_FLUX       <= 0] = np.median(H_BETA_FLUX)
H_ALPHA_FLUX[H_ALPHA_FLUX     <= 0] = np.median(H_ALPHA_FLUX)

# > Calculating diagnostic ratios:
OIII_diagnostic = np.log10(OIII_5007_FLUX / H_BETA_FLUX)  # OIII_5007 / H_beta
NII_diagnostic  = np.log10(NII_6584_FLUX  / H_ALPHA_FLUX) # NII_6584  / H_alpha
SII_diagnostic  = np.log10(SII_6717_FLUX  / H_ALPHA_FLUX) # SII_6717  / H_alpha

# > Organizing data in an analysis-ready fashion:
X = np.stack((OIII_diagnostic,NII_diagnostic,SII_diagnostic),axis=-1)
y = labels

In [59]:
# SPLITTING SAMPLE IN TRAINING, VALIDATION, AND TEST

# Fractions of total sample to be re-distributed into training, validation, and test
train_frac = 0.80
valid_frac = 0.10
test_frac  = 0.10

(X_train, X_remain), (y_train, y_remain) = split_samples(X, y, [train_frac, (valid_frac+test_frac)], random_state=0)
# isolating first the training set ...

(X_valid, X_test), (y_valid, y_test)  = split_samples(X_remain, y_remain, [test_frac, valid_frac], random_state=0)
# ... and now repeting to split validation and training sets

n_train_AGN   = len([y for y in y_train if y == 'true'])
n_train_noAGN = len([y for y in y_train if y == 'false'])

n_valid_AGN   = len([y for y in y_valid if y == 'true'])
n_valid_noAGN = len([y for y in y_valid if y == 'false'])

n_test_AGN    = len([y for y in y_test if y == 'true'])
n_test_noAGN  = len([y for y in y_test if y == 'false'])


print("Sample Summary")
print("________________________________")
print("Total objects    | %-5s" % len(X))
print("-----------------|--------------")
print(" '-> Training    | %-5s" % len(X_train))
print("                 | " + str(X_train.shape))
print("      '-> AGN    | %-5s (%-.1f%%)" % (n_train_AGN,   (n_train_AGN/len(X_train)*100.)))
print("      '-> no AGN | %-5s (%-.1f%%)" % (n_train_noAGN, (n_train_noAGN/len(X_train)*100.)))
print("-----------------|--------------")
print(" '-> Validation  | %-5s" % len(X_valid))
print("                 | " + str(X_valid.shape))
print("      '-> AGN    | %-5s (%.1f%%)" % (n_valid_AGN,   (n_valid_AGN/len(X_valid)*100.)))
print("      '-> no AGN | %-5s (%.1f%%)" % (n_valid_noAGN, (n_valid_noAGN/len(X_valid)*100.)))
print("-----------------|--------------")
print(" '-> Test        | %-5s" % len(X_test))
print("                 | " + str(X_test.shape))
print("      '-> AGN    | %-5s (%.1f%%)" % (n_test_AGN,   (n_test_AGN/len(X_test)*100.)))
print("      '-> no AGN | %-5s (%.1f%%)" % (n_test_noAGN, (n_test_noAGN/len(X_test)*100.)))

Sample Summary
________________________________
Total objects    | 452  
-----------------|--------------
 '-> Training    | 361  
                 | (361, 3)
      '-> AGN    | 34    (9.4%)
      '-> no AGN | 327   (90.6%)
-----------------|--------------
 '-> Validation  | 45   
                 | (45, 3)
      '-> AGN    | 3     (6.7%)
      '-> no AGN | 42    (93.3%)
-----------------|--------------
 '-> Test        | 46   
                 | (46, 3)
      '-> AGN    | 4     (8.7%)
      '-> no AGN | 42    (91.3%)


#### Q: Is ithe sample shaped as expected?
It should be split as < N x M >, where N is the number of objects (with relative fractions given by the train, validation, and test fractions) and M the number of features we can use for the classification.

#### Q: Can we really use the validation/test sample?
No, the statistics are too low. That means for example that we cannot calibrate hyperparameters such as the $k$ in the KNN via cross-validation.

**NOTE:** Addditionally, keep in mind the "demographics" for the training and validation samples (i.e. how many AGNs), and later compare them with the accuracy of each classifier.
If accuracies are similar to the demographics, the classifier is only mirroring the data (i.e. overfitting)!

In [ ]:
# Classifier
model_svc = LinearSVC()
#model_svc = NuSVC()
model_svc.fit(data_train[:,:,5], labels_train)
# NOTE: using only 1 band

# Comparisong with prediction
predicted = model_svc.predict(data_valid[:,:,5])

print("Classification report for %s:\n%s\n"
      % (model_svc, metrics.classification_report(labels_valid, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(labels_valid, predicted))

### 2.1 Loading the necessary packages and data

The data for the example will be retrieved via the `astroML` package (see `astroML.datasets`). From this, we will fetch a sample of SDSS (Sloan Digital Sky Survey) stars including both RR Lyrae and any other type of stars. The `fetch_rrlyrae_combined` function (below) downloads two arrays:
* a $N \times 4$ array for the four colors $g-r$, $u-g$, $r-i$, $i-z$ of $N \approx 100K$ stars
* a $N$-sized array of zeros and ones, where "$1$" is the label which denotes that the respective star is an RR Lyrae

### A technical workaround

As the data to download is about $70$ MB (and may take a while with the public wifi network) and is permenently stored in your computer, we downloaded the data and stored it in a compressed `numpy` format. The function that performs exactly that, is `download_and_save_RRlyrae_data()` and we include it only for future reference.

Now, the script only requires calling `load_data()` and having the file `RRlyrae.npz` in the working directory. In case the latter is missing, just uncomment the call to the function `download_and_save_RRlyrae_data()`.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from astroML.datasets import fetch_rrlyrae_combined
from astroML.utils import split_samples
from astroML.utils import completeness_contamination


PATH = "RRlyrae.npz"


def download_and_save_RRlyrae_data():
    """Create the local file containing photometry and classification of stars in SDSS sample."""
    
    # get data and split into training & testing sets
    print("Getting RR Lyrae data...")
    X, y = fetch_rrlyrae_combined()
    X = X[:, [1, 0, 2, 3]]           # rearrange columns for better plots
    np.savez_compressed(PATH, colors=X, isRR=y)
    print("    Done.")
    
    
def load_data():
    """Load RR Lyrae data from local file."""
    
    print("Loading data from {}...".format(PATH))
    data = np.load(PATH)
    colors, isRR = data["colors"], data["isRR"]
    print("    Done.")
    print("    {:6} objects in total".format(len(isRR)))
    print("    {:6} RR Lyrae in data".format(int(sum(isRR))))
    return colors, isRR


# if the file is missing, uncomment the following line
# download_and_save_RRlyrae_data()

# load photometry and classification (labels) of training data
X, y = load_data()
# X = <Nx4 colors>
# y = <N> labels

# Create boolean "mask" arrays denoting classification as RR Lyrae:
# (will be useful from data exploration/plotting)
isRR = (y == 1)
noRR = (y == 0)

# Limit scatter plots (not histograms) in showing a maximum of 5000 non-RR Lyrae stars:
N_plot = 5000 + int(sum(y))
noRR[:-N_plot] = False

# names of the colors
color_names = ["$g-r$", "$u-g$", "$r-i$", "$i-z$"]

### 2.2. Exploring the data

### Color distributions

We will start easy by creating color histograms for "standard" and RR Lyrae stars to see if they are separable by using just one colour.

In [ ]:
plt.figure(figsize=(12, 8))

for i in range(4):
    color = X[:, i]
    bins = np.linspace(np.nanmin(color), np.nanmax(color), 31)
    
    plt.subplot(221 + i)
    plt.hist(color[isRR],  bins=bins, log=True, color="r", histtype="step", label="RR lyrae")
    plt.hist(color[~isRR], bins=bins, log=True, color="k", histtype="step", label="stars")
    plt.xlabel(color_names[i]  , fontsize=14)
    plt.legend(loc="upper left", fontsize=14)
    
plt.tight_layout()
plt.show()

### A closer 2D look
Maybe using color-color diagrams we can easily classify RR Lyrae.

In [ ]:
plt.figure(figsize=(12, 10))

k = 1
for i in range(4):
    c1 = X[:, i]
    for j in range(i + 1, 4):
        c2 = X[:, j]
        plt.subplot(320 + k)
        plt.plot(c1[noRR], c2[noRR], "k.", label="stars")
        plt.plot(c1[isRR], c2[isRR], "r.", label="RR lyrae")
        plt.xlabel(color_names[i], fontsize=14)
        plt.ylabel(color_names[j], fontsize=14)
        plt.legend(loc="upper right", framealpha=0.9, mode="expand", ncol=2, fontsize=14)
        k += 1
        
plt.tight_layout()
plt.show()

### As close as we can *see* (3D)
What if we plot three colors? Of course we cannot continue with four colors because of our 3D perception limit.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

combinations = [(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)]

fig = plt.figure(figsize=(12, 12))
for index, combination in enumerate(combinations):
    i, j, k = combination
    ax = fig.add_subplot(221 + index, projection='3d')
    ax.view_init(60, -130)  # set camera position for better visualization
    ax.scatter(X[:, i][noRR], X[:, j][noRR], X[:, k][noRR], c=[0.5,0.7,0.7], marker="o", alpha=0.5, edgecolors="k", label="stars")
    ax.scatter(X[:, i][isRR], X[:, j][isRR], X[:, k][isRR], c="r"                                 , edgecolors="k", label="RR Lyrae")
    ax.set_xlabel(color_names[i], fontsize=14)
    ax.set_ylabel(color_names[j], fontsize=14)
    ax.set_zlabel(color_names[k], fontsize=14)
    ax.legend()

plt.show()

The 3D plots do not significantly improve the *manual classification*, compared to the 2D plots. In both cases, we see a *tail* where RR Lyrae variables reside.

### Question
What we would be compromizing if we were to separate RR Lyrae by applying simple horizontal/vertical "cuts" in the 1D, 2D, or 3D color distributions?

### The need for a ML classifier
We can improve over the limitations of the *manual classification* by obtaining:
* mathematical formulation
* automation
* estimates on completeness and contamination of the classification method

## 3. $k$-Nearest Neighbors (KNN) classification

Looking at the scatter plots, we find two blobs corresponding to the different classes. Our mind uses the proximity of the points to form a "mental boundary region" (in a _discriminative classification_ fashion). In this specific case we can try to automate of the procedure, as well as the extension to arbitrary number of dimensions, by mimicking the human behaviour, i.e. by:

> attributing a given point to the class that dominates its surroundings.

The problem is then how to define the "neighborhood" of a point. The trivial solution would be to set a fixed radius. The issue then becomes its size: if too small, we **will not find neighbors** for "satellite" points at the edge of a class cluster; if too large, we **will lose resolution** in dense parts, effectively throwing away information. Therefore, ideally we would like to have a *variable bandwidth* selection threshold.

> One solution is to use a local average of the labels of the $k$ nearest neighbors:
>
> $y = \frac{1}{k} \sum￼_{x_i \in N_k(x)} y_{i}$
>
> where N_k(x) is the neighborhood around $x_i$

In this way the classification _is not_ defined based on the distance on the parameter graph, but is rather scale-independent.

Let's see a 2D example. We got two parameters and training data that are classified as being *red* or *blue*. The question is how do we classify a new (_i.e. not part of the training set_) point? The following images are taken from [3] (we edited the the first one).

### Animated example

<table><tr>
    <td width=400>
        <img src="images/knn_neigh_initial.jpg">
        <center>Figure 3.1.a. Training data already possessing a _red_ or _blue_ label, and an arbitrary new point to be classified.</center>
    </td>    
    <td width=400>
        <img src="images/knn_neigh.gif">
        <center>Figure 3.1.b. Classification using majority votes of $k$ neighbors, fordifferent values of $k$.</center>
    </td>
    <td width=400>
        <img src="images/knn_neigh_mult.gif">
        <center>Figure 3.1.c. For a fixed $k$, the **model** can be thought as of a function of the location in the parameter space. Note that the appearing dots are _not_ part of the training set. Instead, they represent the predicted classifications **if** the new point would fall on that position.</center>
    </td>
</tr></table>

The panels *a* and *b* already suggest that the KNN classification will be affected by the choice of the **hyperparameter** $k$: we will address this issue later on.

### 3.1. Applying $k$-NN on the RR Lyrae photometric data

In [ ]:
# Split the sample in a training [75% of total] and test [25% of total] subsets:
(X_train, X_test), (y_train, y_test) = split_samples(X, y, [0.75, 0.25], random_state=0)

N_tot = len(y)          # number of stars
N_st = np.sum(y == 0)   # number of non-RR Lyrae stars
N_rr = N_tot - N_st     # number of RR Lyrae
N_train = len(y_train)  # size of training sample
N_test = len(y_test)    # size of test sample
N_plot = 5000 + N_rr    # number of stars plotted (for better visualization)
Ncolors = np.arange(1, X.shape[1] + 1)    # number of available colors

print("Sample size")
print("----------------")
print("Total           | %d" % (len(X)))
print(" '-> Train      | %d" % (len(X_train)))
print("     data shape |", ((X_train.shape)))
print(" '-> Test       | %d" % (len(X_test)))
print("     data shape |", ((X_test.shape)))

In [ ]:
# PERFORM CLASSIFICATION FOR VARIOUS VALUES OF k

# for each 'k', store the classifier and predictions on test sample
classifiers = []
predictions = []
kvals = [1, 3, 10] # k values to be used

for k in kvals:
# we try different k hyperparameters

    classifiers.append([])
    predictions.append([])
    
    for nc in Ncolors:
    # for each k, we will use between 1 and 4 colors (1D to 4D) to classify the sources
    
        clf = KNeighborsClassifier(n_neighbors=k) # define the classifier, in this case KNN with hyperparameter k
        clf.fit(X_train[:, :nc], y_train)         # fit training data
        y_pred = clf.predict(X_test[:, :nc])      # predict class of test data

        classifiers[-1].append(clf)
        predictions[-1].append(y_pred)

# Use dedicated astroML to obtain completeness and contamination:
completeness, contamination = completeness_contamination(predictions, y_test)

print("Summary:")
print("--------")

for i,k in enumerate(kvals):
    print("k =", k)
    for nc in Ncolors:
        print("\tno. of colors = %s | " % (nc), end="")
        print("completeness: %.2f " % (completeness[i,nc-1]), end="")
        print("contamination: %.2f" % (contamination[i,nc-1]))

In [ ]:
# COMPUTE AND PLOT DECISION BOUNDARY

import matplotlib as mpl

# classifiers
# <N_k, N_colors>

clf = classifiers[1][1]
# classifier with k=3 and the first 2 colors (g-r, and u-g)
xlim = (-0.15, 0.4) # g-r limits
ylim = (0.7, 1.35)  # u-g limits

# Creating a grid Z of predictions:
xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 71), np.linspace(ylim[0], ylim[1], 81))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Creating a colormap where the RR lyrae (y label = "1") are plotted in red
cmap = mpl.colors.ListedColormap(['black', 'red'])
cmap.set_under('0.5')
cmap.set_over('0.5')

# PLOT THE RESULTS

fig = plt.figure(figsize=(12, 6))
fig.subplots_adjust(bottom=0.15, top=0.95, hspace=0.0, left=0.1, right=0.95, wspace=0.2)

# > left plot: data and decision boundary

ax = fig.add_subplot(121)
im = ax.scatter(X[-N_plot:, 0], X[-N_plot:, 1], c=y[-N_plot:], s=10, lw=0, cmap=cmap, zorder=2)
im.set_clim(-0.5, 1.5)
ax.contour(xx, yy, Z, 1, colors='blue')
# NOTE: The contour showing the locus where a datum is classified as "RR Lyrae" is simply
#       calculated on the grid of predictions
#       To visualize the grid of predictions instead of the contour, use:
#im = ax.imshow(Z, origin='lower', aspect='auto', cmap=cmap, zorder=1, extent=xlim + ylim)
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_title('Decision boundary for classification', fontsize=14)
ax.set_xlabel('$g-r$', fontsize=14)
ax.set_ylabel('$u-g$', fontsize=14)
ax.text(0.02, 0.02, "k = %i" % kvals[1], transform=ax.transAxes, fontsize=14)

# > top-right plot: Completeness vs Ncolors

ax = fig.add_subplot(222)
ax.plot(Ncolors, completeness[0], 'o-k' , ms=6, label='k=%i' % kvals[0])
ax.plot(Ncolors, completeness[1], '^--k', ms=6, label='k=%i' % kvals[1])
ax.plot(Ncolors, completeness[2], 'v:k' , ms=6, label='k=%i' % kvals[2])
ax.xaxis.set_major_locator(plt.MultipleLocator(1))
ax.yaxis.set_major_locator(plt.MultipleLocator(0.2))
ax.xaxis.set_major_formatter(plt.NullFormatter())
ax.set_title('Classifier performance', fontsize=14)
ax.set_ylabel('completeness', fontsize=14)
ax.set_xlim(0.5, 4.5)
ax.set_ylim(-0.1, 1.1)
ax.grid(True)

# > bottom-right plot: contamination vs Ncolors

ax = fig.add_subplot(224)
ax.plot(Ncolors, contamination[0], 'o-k' , label='k=%i' % kvals[0])
ax.plot(Ncolors, contamination[1], '^--k', label='k=%i' % kvals[1])
ax.plot(Ncolors, contamination[2], 'v:k' , label='k=%i' % kvals[2])
ax.legend(loc='lower right', bbox_to_anchor=(1.0, 0.79), fontsize=14)
ax.xaxis.set_major_locator(plt.MultipleLocator(1))
ax.yaxis.set_major_locator(plt.MultipleLocator(0.2))
ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%i'))
ax.set_xlabel('N colors', fontsize=14)
ax.set_ylabel('contamination', fontsize=14)
ax.set_xlim(0.5, 4.5)
ax.set_ylim(-0.1, 1.1)
ax.grid(True)

plt.show()

Note the coarsness of the decision boundary (blue contour), which indicates a potential over-fitting of the training sample.

### 3.2. Choosing the right $k$

### Small $k$ 
> $\large+$ only nearby points are taken into account

> $\large-$ if too small, noisy decision boundaries (see change of classification in Figure 3.1.b for small $k$'s)

### Large $k$
> $\large+$ assuming infinite sample for $k \rightarrow \infty$, the *classification error rate* is minimized

> $\large-$ **but** real-life samples are finite, so large $k$ results to loss of resolution (over-smoothing)

### Some approaches

> take $k = \sqrt{N}$

> use *cross-validation* to select optimal $k$

> if $2$ classes, go for an odd $k$ to avoid ties

Figure 3.2 (from [4]) reports an example of over-smoothing.

<table>
    <tr>
        <td><img src="images/k1.jpg"   width=400></td>    
        <td><img src="images/k5.jpg"   width=400></td>
        <td><img src="images/k399.jpg" width=400></td>
    </tr>  
    <tr>
        <center>
            Figure 3.2. Variation of classification boundaries as $k$ increases to very large values.
        </center>    
    </tr>
</table>

### 3.3. Defining the neighborhood
What does "near" in "nearest neighbors" mean?
Identifying neighbors requires calculating a **distance** between points or, better said, define a **metric**. A metric might be difficult to define or might not be definable at all for the type of features we are inspecting (e.g. imagine classifying galaxies by morphology and color). Consider for example the following issues.

* One simple choice would be to adopt the common Euclidean metric. However that would not be performing properly when the parameter spans are different (e.g. one parameter has span $\left[1, 100\right]$ while the other has span $\left[0, 1\right]$). This might require **feature weighting** (normalization).
* What if all or some characteristics are not numeric but categorial? What is the metric in this case (see Hamming distance)?

For more information, consult [5].

### 3.4. Weighted KNN ($w$KNN)
We can mitigate the impact of choosing the appropriate $k$ by imposing that each vote on the class is weighted by the distance (or *similarity*) from the point to be classified. In this case, the classification algorithm for point $x$ becomes:

> $y = \frac{1}{k} \sum￼_{x_i \in N_k(x)} ~K(d) \times y_{i}$
>
> where $d$ is the distance and $K(d)$ is the **kernel** which converts the distance into a weight. One simple example is the inversion kernel $K(d) = \frac{1}{d}$

For an extensive treatment, consult [6].

### 3.5. Final remarks on KNN

### Pros
* No need to assume distribution (_discriminative classifier_)
* Simple and intuitive
* Robust for large samples

### Cons
* Hard to select $k$
* Computationally expensive: $O(nkd)$, where $n$ is the size of training sets, and $d$ is the dimension of each training set (but can be optimised)
* Good accuracy requires large samples unifromly covering the parameter space

## 4. Support Vector Machine (SVM)

Support vector machine (SVM) is a way of choosing a linear decision boundary between different classes.

The classification boundary is provided by the hyperplane maximizing the distance between the hyperplane itself and the closest point from either class. This distance is called *margin*. Points on the margins are called *support vectors*.

<table><tr>
    <td width=400>
        <img src="images/SVM_1.png">
        <center>Figure 4.1.a. Hyperplane (dashed line) separating two classes (_red_ and _green_).</center>
    </td>    
    <td width=400>
        <img src="images/SVM_2.png">
        <center>Figure 4.1.b. The closest points to the hyperplane from each class constitute the "tip" of the *support vectors*.</center>
    </td>
</tr></table>

Figure 4.1.a shows two different classes (e.g. young and old stars) distributing in a scatter plot according to variable $x_1$ (e.g. radius) and $x_2$ (e.g. temperature). Figure 4.1.b explains the origin of the name "support vectors": the closest points _support_ the hyperplanes (solid lines) equally distant from the decision hyperplane (dashed line).

Infinite possible boundaries can separate the two classes. SVM algorithms find the one that maximizes the distance between the supported hyperplanes.

### 4.1. Hyperplanes and decision boundary

The supported hyperplanes (solid-lines in Figure 4.1.a can be defined as:

> **w**$\cdot$**x** + b = +1
>
> **w**$\cdot$**x** + b = -1

where **x** is the coordinate on the (x1, x2) plane, **w** is a 2$\times$1 matrix and **b** a scalar. It turns out that these hyperplanes are separated by a distance 2 / ||w||. Finding the ideal classification boundary, i.e. the one maximizing the distance, is therefore a problem of minimizing the norm ||w||. This is what SVM algorithms do.

For an complete mathematical formulation, consult [7].

### 4.2. Separatable classes (or not)

We cannot always assume that 2 classes are separble without "contamination". That is why SVM algorithms include **slack variables**, a tunable parameter which penalizes misclassifications.


### 4.3. Multiple classes

The SVM method can be applied for multiple classes as well (Figure 4.3).

<img src="images/svm_many_classes.png" width=400>
<center>
    Figure 4.3: SVM applied to 3 different classes.
</center>

### 4.4. Multiple dimensions

If our sample of stars is characterized by three parameters (X, Y, Z), e.g. radius, temperature and mass, then the scatter plot has 3 dimensions (Figure 4.3). The boundary between the classes in the 3-D plot is a plane. Because of the fact that the method can be extrapolated at N-dimensions, the boundary is a *hyperplane*.

<img src="images/svm_3d.png" width=400>
<center>
    Figure 4.4: Support vector machine applied for 3-D features and three classes.
</center>

### 4.5. Applying linear SVM to the RR Lyrae photometric data

We will use SVM to our RR Lyrae example to separate objects with a linear decision boundary.
In $\S$4.6 and $\S$4.7 we will see an extension of SVM adopting non-linear boundaries.

In [ ]:
# In this block, we define a function that applies SVM on our data.
# If the boolean argument is set to `True`, it applies the linear SVM (this section)
# otherwise it uses Gaussian Radial Basis function kernel (RBF; Section 4.7).

def apply_SVM(linear):
    if linear:
        kernel_to_use = "linear"
        gamma_to_use = "auto"
    else:
        kernel_to_use = "rbf"
        gamma_to_use = 20.0

    def compute_SVM(Ncolors):
        classifiers = []
        predictions = []
        
        for nc in Ncolors:
            print("    Computing for", nc, "color(s)...")
            # perform support vector classification
            clf = SVC(kernel=kernel_to_use, gamma=gamma_to_use, class_weight='balanced')
            clf.fit(X_train[:, :nc], y_train)
            y_pred = clf.predict(X_test[:, :nc])

            classifiers.append(clf)
            predictions.append(y_pred)

        return classifiers, predictions

    print("Performing SVM classification...")

    classifiers, predictions = compute_SVM(Ncolors)

    completeness, contamination = completeness_contamination(predictions, y_test)

    print("completeness",  completeness)
    print("contamination", contamination)

    # COMPUTE THE DECISION BOUNDARY
    
    """
    NOTE: The sklearn SVM routine does _not_ return the coefficients of the
          decision boundary.
          For example, for a 2D case where the decision boundary is a line
          of the type:
    
             y = a*x + b
    
          one would expect to obtain "a" and "b" (where b == intercept).
          Instead, the w matrix is returned (see Section 4.1).
          In the 2D case, for a linear model, the conversion is trivial
          (see the code below for a linear case).
          
          For a more generic case of representing a curved boundary in N
          dimensions, it is more convenient to generate a grid ("Z") of
          predictions covering the whole parameter space (essentially a map
          of the classes), and then plot the contour around a class
          (see the code below for a non-linear case).
          
          Alternatively, one can use the sklearn built-in function to
          draw the decision function:

              https://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane.html
    """
    
    clf = classifiers[1]
    # loading classifier for 2 colors

    if linear:
        # > Extracting slope and intercept of boundary line:
        w = clf.coef_[0]
        a = -w[0] / w[1]
        yy = np.linspace(-0.1, 0.4)
        xx = a * yy - clf.intercept_[0] / w[1]
    else:
        # > Creating a grid of predictions:
        xlim = (0.7, 1.35)
        ylim = (-0.15, 0.4)
        xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 101), np.linspace(ylim[0], ylim[1], 101))
        Z = clf.predict(np.c_[yy.ravel(), xx.ravel()])
        Z = Z.reshape(xx.shape)
        # Smooth the boundary:
        from scipy.ndimage import gaussian_filter
        Z = gaussian_filter(Z, 2)

    # PLOT THE RESULTS

    fig = plt.figure(figsize=(12, 6))
    fig.subplots_adjust(bottom=0.15, top=0.95, hspace=0.0, left=0.1, right=0.95, wspace=0.2)

    # left plot: data and decision boundary
    ax = fig.add_subplot(121)
    im = ax.scatter(X[-N_plot:, 1], X[-N_plot:, 0], c=y[-N_plot:], s=4, lw=0, cmap=plt.cm.binary, zorder=2)
    if linear:
        ax.plot(xx, yy, '-k')
    else:
        ax.contour(xx, yy, Z, [0.5], colors='k')

    
    im.set_clim(-0.5, 1)
    ax.set_xlim(0.7, 1.35)
    ax.set_ylim(-0.15, 0.4)
    ax.set_xlabel('$u-g$', fontsize=14)
    ax.set_ylabel('$g-r$', fontsize=14)

    # plot completeness vs Ncolors
    ax = fig.add_subplot(222)
    ax.plot(Ncolors, completeness, 'o-k', ms=6)
    ax.xaxis.set_major_locator(plt.MultipleLocator(1))
    ax.yaxis.set_major_locator(plt.MultipleLocator(0.2))
    ax.xaxis.set_major_formatter(plt.NullFormatter())
    ax.set_ylabel('completeness', fontsize=14)
    ax.set_xlim(0.7, 4.5)
    ax.set_ylim(0.7, 1.1)
    ax.grid(True)

    # plot contamination vs Ncolors
    ax = fig.add_subplot(224)
    ax.plot(Ncolors, contamination, 'o-k', ms=6)
    ax.xaxis.set_major_locator(plt.MultipleLocator(1))
    ax.yaxis.set_major_locator(plt.MultipleLocator(0.2))
    ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%i'))
    ax.set_xlabel('N colors',      fontsize=14)
    ax.set_ylabel('contamination', fontsize=14)
    ax.set_xlim(0.7, 4.5)
    ax.set_ylim(0.7, 1.1)
    ax.grid(True)

    plt.show()

In [ ]:
apply_SVM(linear=True)

### 4.6. Non-linear boundaries

Sometimes, linear boundaries may not be optimal and a non-linear SVM should be used instead. The left panel of Figure 4.6 shows an 2D scatter plot of two different classes (e.g. red and green stars with different radii and temperatures) which cannot be linearly separated.

In order to find non-linear boundaries we can tackle the problem in an higher dimensional space. We use a process called **kernelization**, which consists in using a kernel function to attribute to our data a value in the additional dimension. Then, we draw the decision hyperplane into this higher dimensional space.

The central panel of Figure 4.6 shows that once the 2D data are mapped to a 3D space by attributing a $z$ value through a Gaussian-like function, the classes are easily separable by a 3D hyperplane. Projecting back the plane in 2D, we obtain the non-linear boundary (Figure 4.6, rght panel).

<img src="images/kernel.png" width=800>
<center>
    Figure 4.6. When no linear boundaries can be used the SVM method can be applied by using kernel.
</center>

### Choosing the kernel function

Useful kernel functions shall satisfy specific conditions (see [7]), so that in practice only a few are used. In the example of Figure 4.6, the Gaussian Radial Basis Function is used:

> $K(x,y) = e^{-\gamma(x-y)^2}$

where $\gamma$ is a hyperparameter which shall be learned via cross-validation (in our example we use an arbitrary value).

### 4.7. Applying kernelized SVM to the RR Lyrae photometric data

In [ ]:
apply_SVM(linear=False)

### 4.8. Misclassifications

Figure 4.3 shows that the blue points class is contaminated by some green 
points (*misclassified* points). Likewise, the green points class is contaminated by blue points. This contamination makes it difficult to define the boundary between the two classes.

In this situation SVM finds the hyperplane that maximizes the margin and indirectly minimizes the misclassifications. However SVM is not designed to minimize the contamination _per se_.


### 4.9. Final remarks on SVM

### Pros
* Good at dealing with high dimensional data
* Works well on small data sets

### Cons
* Picking the right kernel and parameters can be computationally intensive
* It suffers from contamination


For further information on SVM, see [8].

## References
[1] http://esoads.eso.org/abs/2003LNP...635...85B

[2] https://www.eso.org/sci/publications/messenger/archive/no.13-jun78/messenger-no13-15-17.pdf

[3] https://importq.wordpress.com/2017/11/24/mnist-analysis-using-knn/

[4] https://idc9.github.io/stor390/notes/cross_validation/cross_validation.html

[5] http://www.cs.haifa.ac.il/~rita/ml_course/lectures/KNN.pdf

[6] https://epub.ub.uni-muenchen.de/1769/1/paper_399.pdf

[7] http://web.mit.edu/6.034/wwwbob/svm-notes-long-08.pdf

[8] http://www.saedsayad.com/support_vector_machine.htm